In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

In [ ]:
!pip install accelerate

In [ ]:
!pip install duckduckgo-search

## Load Model weights

In [ ]:
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

In [ ]:
model_id = "google/gemma-3-4b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
torch.cuda.empty_cache()

## Function Calling Gemma

In [ ]:
from duckduckgo_search import DDGS

def search(query:str):
  """
  search results to the user query

  Args:
      query: user prompt to fetch search results
  """
  req = DDGS()
  response = req.text(query,max_results=4)
  context = ""
  for result in response:
    context += result['body']
  return context

In [ ]:
tools = [search]

In [ ]:
query = "which teams played Cricket ICC Champions Trophy 2025 finals and who won?"

In [ ]:
conversation = [
    {
        "role": "system",
        "content": [{"type": "text", "text": """
        You are an expert search assistant. Use `search` to get the most accurate details.
        At each turn, if you decide to invoke any of the function(s), it should be wrapped with ```tool_code```. The python methods described below are imported and available,
        you can only use defined methods. The generated code should be readable and efficient. The response to a method will be wrapped in ```tool_output``` use it to call more tools or generate a helpful, friendly response.
        When using a ```tool_call``` think step by step why and how it should be used.

        The following Python methods are available:
        \`\`\`python
        def search(query:str):
          "
          search results to the user query

          Args:
             query: user prompt to fetch search results
          "
        \`\`\`

        User: \{user_message\}
        """}]
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": query}
        ]
    }
]

In [ ]:
inputs = processor.apply_chat_template(
            conversation,
            tools=tools,
            add_generation_prompt=True,
            return_dict=True,
            tokenize=True,
            return_tensors="pt",
).to(model.device, dtype=torch.bfloat16)

In [ ]:
outputs = model.generate(**inputs, max_new_tokens=512)

In [ ]:
output = processor.decode(outputs[0], skip_special_tokens=True)

In [ ]:
response = output.split("model")[-1]

In [ ]:
print(response)


```tool_code
search(query="Which teams played in the Cricket ICC Champions Trophy 2025 final and who won?")
```


In [ ]:
def extract_tool_call(text):
    import io
    from contextlib import redirect_stdout

    pattern = r"```tool_code\s*(.*?)\s*```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        code = match.group(1).strip()
        # Capture stdout in a string buffer
        f = io.StringIO()
        with redirect_stdout(f):
            result = eval(code)
        output = f.getvalue()
        r = result if output == '' else output
        return f'```tool_output\n{r}\n```'''
    return None

In [ ]:
keyword = extract_tool_call(response)

In [ ]:
keyword

"```tool_output\nThe ninth edition of the Champions Trophy 2025 saw India being crowned as the winners on 9th March 2025 after they overcame New Zealand in the final. Several exceptional performers lit up the tournament with the bat and ball. The best of them made it to the Team of the Tournament. Here's what the side looks like: 1. Rachin Ravindra (New Zealand)The 2025 ICC Champions Trophy was the ninth edition of the ICC Champions Trophy, a quadrennial ODI cricket tournament organised by the International Cricket Council (ICC). In November 2021 as part of the 2024-2031 ICC men's hosts cycle, the ICC announced that the 2025 Champions Trophy would be played in Pakistan. [4]On 19 December 2024, following an agreement between Board of Control for ...Official ICC Champions Trophy, 2025 Cricket website - live matches, scores, news, highlights, commentary, rankings, videos and fixtures from the International Cricket Council. ... 2025. ICC announces Champions Trophy 2025 Team of the Tourname

In [ ]:
prompt = [{
        "role": "user",
        "content": [
            {"type": "text", "text": f"Results from the search:{keyword} User_query : {query}"}
        ]
    }
]

In [ ]:
final_prompt = processor.apply_chat_template(
            prompt,
            tools=tools,
            add_generation_prompt=True,
            return_dict=True,
            tokenize=True,
            return_tensors="pt",
).to(model.device, dtype=torch.bfloat16)

In [ ]:
response = model.generate(**final_prompt, max_new_tokens=512)

In [ ]:
print(processor.decode(response[0], skip_special_tokens=True))

user
Results from the search:```tool_output
The ninth edition of the Champions Trophy 2025 saw India being crowned as the winners on 9th March 2025 after they overcame New Zealand in the final. Several exceptional performers lit up the tournament with the bat and ball. The best of them made it to the Team of the Tournament. Here's what the side looks like: 1. Rachin Ravindra (New Zealand)The 2025 ICC Champions Trophy was the ninth edition of the ICC Champions Trophy, a quadrennial ODI cricket tournament organised by the International Cricket Council (ICC). In November 2021 as part of the 2024-2031 ICC men's hosts cycle, the ICC announced that the 2025 Champions Trophy would be played in Pakistan. [4]On 19 December 2024, following an agreement between Board of Control for ...Official ICC Champions Trophy, 2025 Cricket website - live matches, scores, news, highlights, commentary, rankings, videos and fixtures from the International Cricket Council. ... 2025. ICC announces Champions Troph